In [2]:
import numpy as np
import pandas as pd
import os

### Download videos for training

Downloads videos as mp4 and saves the audio as mp3. mp4 may or may not contain audio due to the API being limited, though this should not matter (except for the small negative impact on file size). Currently videos are being downloaded in 720p since 1080p is inconsistent/bugged.

In [5]:
from pytube import YouTube 
  
urls = ["https://www.youtube.com/watch?v=Yt3-a9mExZg", "https://www.youtube.com/watch?v=cGt8bEcd9Ms", 'https://www.youtube.com/watch?v=w00ooES3D5Y']
urls = ['https://www.youtube.com/watch?v=w00ooES3D5Y']
pathVideo = 'Data/Video'
pathAudio = 'Data/Audio'

for i, url in enumerate(urls):
    print('yo')
    yt = YouTube(url)
    yt.streams.filter(abr="160kbps", progressive=False).first().download(filename="Audio"+str(i)+".mp3", output_path = pathAudio)
    yt.streams.filter(res="720p", progressive=False).first().download(filename="Video"+str(i)+".mp4", output_path= pathVideo)
##

yo


AttributeError: 'NoneType' object has no attribute 'span'

### Split video into frames

Goes through each downloaded video. Creates a directory per video inside 'tempFrames' and then saves the frames here. We can set the amount of frames we want per minute. Note that file sizes of these images can balloon quickly, exceeding the filesize of the video itself.

This code is mainly for early testing

In [10]:
import cv2
FRAMES_PER_MINUTE = 10
frame_counter = 60000 / FRAMES_PER_MINUTE
pathVideo = 'Data/Video/'
videoNames = []
for filename in os.listdir(pathVideo):
    f = os.path.join(pathVideo, filename)
    if os.path.isfile(f):
        if filename != '.gitignore':
            videoNames.append(filename)

for video in videoNames:
    path = pathVideo + video
    outputPath = 'Data/tempFrames/' + video[:-4] + '/' ## make directory per video
    if not os.path.exists(outputPath):
        os.mkdir(outputPath)
        print(f'Created new dir {outputPath}')
    vidcap = cv2.VideoCapture(pathVideo + video)
    success,image = vidcap.read()
    print(f'Succesful videocapture?: {success}')
    count = 0
    while success:
        cv2.imwrite(outputPath + "%d_seconds.jpg" % (count*(frame_counter/1000)), image)     # save frame as JPEG file   
        count += 1
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*frame_counter))   
        success,image = vidcap.read()
        

Succesful videocapture?: True
Succesful videocapture?: True


### Py-Feat

This library provides a pipeline containing multiple models related to facial detection. The most interesting aspect here is the output of AU's (action units), which will be a key part of predicting interest levels.

While the library works okay, it has difficulties with cuda and the documentation is lacking. The goal here is to use feat-py for prototyping and then later create a similar pipeline myself. This gives me more possibilities in for example fine-tuning, while also giving more control with regards to memory management and gpu usage. 

Action units cheat-sheet: https://py-feat.org/pages/au_reference.html

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import feat
from feat.facepose_detectors.img2pose.img2pose_test import Img2Pose
import torch
from torchvision import transforms
from PIL import Image
from Models.tracking_persistence.face_tracking_persistence import face_persistence_model
from Preprocessing.videoCropper import videoCropper
convert_tensor = transforms.ToTensor()

In [12]:
from feat.utils.io import get_test_data_path
from feat.plotting import imshow

# Helper to point to the test data folder
test_data_dir = get_test_data_path()

# Get the full path
single_face_img_path = os.path.join(test_data_dir, "single_face.jpg")

#single_face_prediction = detector.detect_image(single_face_img_path)
single_face_prediction = detector.detect_image('Data/testImages/ac3.jpg')

# Show results
single_face_prediction.plot_detections(au_barplot=False, emotion_barplot=False)
single_face_prediction

NameError: name 'detector' is not defined

In [1]:
from feat.utils.io import get_test_data_path
import os

test_data_dir = get_test_data_path()
test_video_path = os.path.join(test_data_dir, "WolfgangLanger_Pexels.mp4")
# test_video_path = "Data/Video/Video1.mp4"

# Show video
# from IPython.core.display import Video
# Video(test_video_path, embed=False)

In [17]:
import openface

ModuleNotFoundError: No module named 'openface'

In [ ]:
video_prediction

##### Face-tracker
The class `face_persistence_model` is based on the `face_recognition` library, using the `face_locations` and `face_encoding` functions. `face_locations` searches a frame for faces and returns bounding box coordinates. `face_encoding` checks these faces to see if they can recognize the person. When it encounters a "new" face it saves some metadata. Then when detecting future faces it will check this new face against all the metadata files.

Since the `face_persistence_model` serves as both a face detection and recognition model, it returns a dataframe consisting of multiple rows per frame, one row per detected face. The columns are `['Frame', 'Person', 'FaceRectX', 'FaceRectY', 'FaceRectWidth', 'FaceRectHeight']`. 

The inputs for the model are:
1. `TOLERANCE`: A number [0,1] that alters how the model recognizes faces. A too low tolerance leads to the same face getting detected as different unique faces, too high tolerance leads to multiple faces being detected as the same person 
2. `filename`: The name of the videofile being analyzed. Must contain extension such as ".mp4".
3. `frameSkips`: This integer value represents how often a frame is skipped, i.e. if `frameSkips=3` every 3rd frame is skipped.
4. `model`: Face location model choice of `['cnn', 'hog']`. While 'hog' is quicker (especially on CPU), 'cnn' is more accurate and utilizes GPU.
5. `modelSize`: Size of the face encoding model, choice of `['small', 'large']`.


In [2]:
from Models.tracking_persistence.face_tracking_persistence import face_persistence_model
from Models.face_detection.face_detection import face_detector

filename = 'Video2_Trim_Trim.mp4'

face_detector_model = face_detector(filename=filename, detection_threshold=0.25,)
output = face_detector_model.runModel()
face_tracker = face_persistence_model(TOLERANCE=0.6, filename=filename, model='cnn', modelSize='large')
output = face_tracker.runDetectionTrackOnly(output, True)
face_tracker.saveCSV(output)
videoCrop = videoCropper(output, filename, displayVideo=False, saveCSV=True)
output = videoCrop.runModel()
videoCrop.saveCSV()

KeyboardInterrupt: 

Video2_Trim_Trim.mp4


In [2]:
import numpy as np
import pandas as pd
from Models.tracking_persistence.face_tracking_persistence import face_persistence_model
from Models.face_detection.face_detection import face_detector
from feat.au_detectors.StatLearning.SL_test import XGBClassifier
filename = 'Video2_Trim_Trim.mp4'
face_detector_model = face_detector(filename=filename, detection_threshold=0.25,)
output = face_detector_model.runModel()
face_tracker = face_persistence_model(TOLERANCE=0.6, filename=filename, model='cnn', modelSize='large')
output = face_tracker.runDetectionTrackOnly(output, False)
face_tracker.saveCSV(output)


Video loaded? :  True


In [37]:
print(top)
(output['FaceRectY'].value_counts())
(df['FaceRectY'].value_counts()[459-top])


3


42

### Video segmentation

To reduce memory usage it could be worthwile to only work with the video data inside of bounding boxes. I experimented with this in face_tracking_persistency.py

In [8]:
initialDF[1307:1313]

,Frame,Person,FaceRectX,FaceRectY,FaceRectWidth,FaceRectHeight
1307,438,0,888,40,94,95
1308,438,2,271,138,113,114
1309,438,1,547,472,114,114
1310,439,0,888,40,94,95
1311,439,2,259,138,114,114
1312,439,1,547,472,114,114


In [10]:
import numpy as np
import cv2
from feat import Detector
from feat.au_detectors.StatLearning.SL_test import XGBClassifier
from feat.emo_detectors.StatLearning.EmoSL_test import EmoSVMClassifier

mydet = Detector(emotion_model='svm')

x_columns = [f'x{i}' for i in range(68)]
y_columns = [f'y{i}' for i in range(68)]
face_columns = ['FaceRectWidth', 'FaceRectHeight']
bbox_columns = ['FaceRectX', 'FaceRectWidth', 'FaceRectY', 'FaceRectHeight']
au_keys = ["AU1","AU2","AU4","AU5","AU6","AU7","AU9","AU10","AU11","AU12","AU14",
           "AU15","AU17","AU20","AU23","AU24","AU25","AU26","AU28","AU43"]
emotion_keys = ['anger', 'disgust', 'fear',	'happiness', 'sadness',	'surprise',	'neutral']
auDF = pd.DataFrame(index=range(len(output)), columns=au_keys)
emoDF = pd.DataFrame(index=range(len(output)), columns=emotion_keys)
video = cv2.VideoCapture('./Data/Video/'+filename)
frame_i = 0
total_iterator = 0

while True:
    ret, image = video.read()
    if ret == False:
        break
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Assuming your DataFrame is named 'df'
    grouped = output.groupby('Frame')
    for frame, group in grouped:
        if frame == frame_i:
            frame_group = group
            break
    ## Gather the rows of the current frame
    #tempAUdf = pd.DataFrame(index=range(len(frame_group)), columns=au_keys)
    #tempEmo = pd.DataFrame(index=range(len(frame_group)), columns=emotion_keys)
    # Fill the DataFrame with empty values
    tempRowList = []
    for _, row in frame_group.iterrows():
        #face = row[bbox_columns].tolist()
        
        x_array = row[x_columns].tolist()
        y_array = row[y_columns].tolist()
        coordinates = list(zip(x_array, y_array))
        coordinates = np.array(coordinates, dtype=np.float32)
        nested_coordinates = [[coordinates]]
        aus = mydet.detect_aus(image, nested_coordinates)
        emotions = mydet.detect_emotions(frame=image, facebox=['a','b'], landmarks=nested_coordinates)
        for name, key in zip(au_keys,aus[0][0]):
            auDF.loc[total_iterator, name] = key
        for name, key in zip(emotion_keys, emotions[0][0]):
            emoDF.loc[total_iterator, name] = key
        total_iterator += 1
    
    #preds = aumodel.detect_au('frame', landmarks)
output2 = output.copy(deep=True)
output2 = pd.concat([output2, auDF, emoDF], axis=1)
auDF[:6]


,AU1,AU2,AU4,AU5,AU6,AU7,AU9,AU10,AU11,AU12,AU14,AU15,AU17,AU20,AU23,AU24,AU25,AU26,AU28,AU43
0,0.645077,0.544905,0.540408,0.403355,0.094322,0.0,0.312239,0.257027,0.462895,0.0516,0.099903,0.569102,0.551997,0.0,0.460705,0.462217,0.009313,0.07818,0.094837,0.143593
1,0.322274,0.443248,0.561686,0.419202,0.163491,1.0,0.265812,0.475733,0.501563,0.277127,0.657337,0.183199,0.517525,0.0,0.484364,0.562959,0.005042,0.128643,0.639673,0.057386
2,0.440837,0.370957,0.652599,0.409511,0.151656,0.0,0.235934,0.010426,0.470701,0.067009,0.315489,0.249759,0.554241,0.0,0.286486,0.461533,0.005106,0.243127,0.035802,0.104062
3,0.632171,0.481221,0.406721,0.359588,0.105413,1.0,0.233528,0.139306,0.460302,0.044087,0.095755,0.419754,0.5208,0.0,0.26592,0.47535,0.036728,0.06679,0.072695,0.255903
4,0.308672,0.450978,0.540531,0.425464,0.163491,0.0,0.286017,0.29592,0.500874,0.257023,0.719159,0.240338,0.518099,0.0,0.539826,0.560742,0.007452,0.16884,0.565133,0.057386
5,0.386172,0.360003,0.552023,0.413747,0.151656,0.0,0.239818,0.000803,0.471212,0.062359,0.39728,0.221722,0.542388,0.0,0.285394,0.461533,0.006028,0.245171,0.063414,0.068021


In [11]:
output2[:10]

,Frame,FaceRectX,FaceRectY,FaceRectWidth,FaceRectHeight,pose1,pose2,pose3,Person,x0,...,AU26,AU28,AU43,anger,disgust,fear,happiness,sadness,surprise,neutral
0,0,567.0,432.0,129.0,156.0,6.467462,-1.903672,-2.535942,0,587,...,0.07818,0.094837,0.143593,0,1,1,0,1,1,1
1,0,878.0,7.0,142.0,167.0,-17.975130,-5.846595,4.860634,1,902,...,0.128643,0.639673,0.057386,0,0,0,0,0,0,1
2,0,286.0,127.0,115.0,137.0,9.228345,1.559841,-6.363375,2,305,...,0.243127,0.035802,0.104062,0,1,0,0,0,0,1
3,1,567.0,432.0,129.0,156.0,4.913262,-2.646218,-2.945271,0,586,...,0.06679,0.072695,0.255903,0,1,1,0,1,1,1
4,1,878.0,7.0,142.0,167.0,-17.947324,-5.862958,4.847039,1,901,...,0.16884,0.565133,0.057386,0,0,0,0,0,0,1
5,1,286.0,127.0,115.0,137.0,9.234969,1.568362,-6.341674,2,306,...,0.245171,0.063414,0.068021,0,1,0,0,0,0,1
6,2,565.0,433.0,131.0,161.0,4.313297,-2.944048,-3.572917,0,587,...,0.094174,0.189,0.189185,0,1,1,0,1,1,1
7,2,880.0,5.0,142.0,171.0,-16.650138,-6.082811,7.687734,1,901,...,0.153952,0.445016,0.065685,0,0,0,0,0,0,1
8,2,288.0,129.0,111.0,128.0,10.257100,1.564581,-6.734790,2,303,...,0.134778,0.022374,0.067519,0,1,0,0,0,0,1
9,3,565.0,433.0,131.0,161.0,4.372026,-2.911112,-3.595962,0,586,...,0.077004,0.069421,0.191711,0,1,1,0,1,1,1
